# U.S. Medical Insurance Costs

Goal: The goal of this project is to optimize the insurance cost in my future. This project will help me decide where to live if I want to have children and what my bmi should be (regarding bmi I will try to inspect if lower means alwas better). 

Data: For this project I am assuming that insurance is provided by one company. Also In the main part I won't focus on sex and smoking since I want change my sex and I will never smoke. So I will filter them out

Firstly I have imported pandas library and insurance.csv

In [ ]:
import csv
import pandas as pd

starting_df = pd.read_csv("insurance.csv")
test = 1111



Now I want to check if there are any rows with Null or other "bad" values.

In [15]:

def check_for_nulls (df):
    df["nulls"] = df.apply(lambda row: row.isnull().any(), axis=1)
    return df
test_null = check_for_nulls(starting_df)

print(test_null["nulls"].unique())



[False]


In [16]:
starting_df.drop(columns=['nulls'], inplace=True)
print(starting_df.columns)

Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges'], dtype='object')
